In [ ]:
!pip install deepface
!pip install easyocr
!pip install gradio
!pip install regex

In [ ]:
import gradio as gr
import easyocr
import numpy as np
import regex as re
from datetime import datetime
from deepface import DeepFace
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])

In [ ]:
# STEP 3: DATA PARSING FUNCTION (Unchanged)
def parse_kyc_details(text):
    details = {
        "Name": "Not found",
        "Document Number": "Not found",
        "Date Of Birth": "Not found",
        "Issue Date": "Not found",
        "Address": "Not found",
        "Nationality": "NEPALESE"
    }
    name_match = re.search(r'Full Name\s*\n\s*([A-Z\s]+?)\s*(\n|$)', text, re.IGNORECASE)
    if name_match:
        details["Name"] = name_match.group(1).strip()
    doc_num_match = re.search(r'Certificate No\s*[:.\s]*\n?([\d-]+)', text, re.IGNORECASE)
    if doc_num_match:
        details["Document Number"] = doc_num_match.group(1).strip()
    dob_match = re.search(r'Year\s*:\s*(\d{4})\s*Month\s*:\s*([A-Z]{3})\s*Day\s*\.?\s*(\d{1,2})', text, re.IGNORECASE)
    if dob_match:
        year, month, day = dob_match.groups()
        details["Date Of Birth"] = f"{year}-{month}-{day}"
    issue_date_match = re.search(r'(\d{4}[-.]\d{2}[-.]\d{2})', text)
    if issue_date_match:
        details["Issue Date"] = issue_date_match.group(1).strip()
    address_match = re.search(r'Permanent Address[\s\S]*?District\s*:\s*([A-Za-z]+)', text, re.IGNORECASE)
    if address_match:
        details["Address"] = "District: " + address_match.group(1).strip()
    return details

In [ ]:
def verify_faces(id_card_image, selfie_image):
    """
    Compares faces using the DeepFace library.
    """
    try:

        result = DeepFace.verify(
            img1_path = id_card_image,
            img2_path = selfie_image,
            model_name = 'VGG-Face',
            enforce_detection = False
        )

        # Check the result from the dictionary
        if result['verified']:
            return "Faces Match!", True
        else:
            # Check if faces were even detected
            if result['face_confidence'] < 0.5:
                 return "Face not clearly detected in one or both images.", False
            return " Faces Do Not Match!", False

    except Exception as e:
        return f"CRITICAL ERROR during face verification: {str(e)}", False

def extract_text_from_id(id_card_image):
    results = reader.readtext(id_card_image)
    return "\n".join([result[1] for result in results])

def calculate_fraud_score(parsed_details, face_match_status):
    score = 0
    indicators = []
    missing_fields = [key for key, value in parsed_details.items() if value == "Not found"]
    if missing_fields:
        score += len(missing_fields) * 15
        indicators.append(f"Could not extract: {', '.join(missing_fields)}")
    if not face_match_status:
        score += 40
        indicators.append("Face verification failed or faces did not match.")
    score = min(score, 100)
    risk_level = "🟢 Low"
    if 30 <= score < 60:
        risk_level = "🟡 Medium"
    elif score >= 60:
        risk_level = "🔴 High"
    return score, risk_level, indicators

In [ ]:
# MAIN ORCHESTRATOR FUNCTION
def kyc_agent(id_card, selfie):
    if id_card is None or selfie is None:
        return "Please upload both an ID card and a selfie."

    raw_text = extract_text_from_id(id_card)
    parsed_details = parse_kyc_details(raw_text)
    face_message, faces_match = verify_faces(id_card, selfie)
    fraud_score, risk_level, fraud_indicators = calculate_fraud_score(parsed_details, faces_match)

    status = "VERIFIED"
    if "Not found" in parsed_details.values() or not faces_match:
        status = "⚠️ MANUAL REVIEW REQUIRED"
    if "CRITICAL ERROR" in face_message:
        status = "PROCESSING ERROR"

    report = f"""
KYC Verification Results
Status: {status}
Timestamp: {datetime.now().isoformat()}

📄 Extracted Document Data
Name: {parsed_details['Name']}
Document Number: {parsed_details['Document Number']}
Date Of Birth: {parsed_details['Date Of Birth']}
Issue Date: {parsed_details['Issue Date']}
Address: {parsed_details['Address']}
Nationality: {parsed_details['Nationality']}

👤 Face Verification
Message: {face_message}

🔍 Fraud Detection
Fraud Score: {fraud_score}/100
Risk Level: {risk_level}
Indicators:
- {"\n- ".join(fraud_indicators) if fraud_indicators else "No significant risk indicators found."}
"""
    return report

In [ ]:
#GRADIO INTERFACE
iface = gr.Interface(
    fn=kyc_agent,
    inputs=[
        gr.Image(type="numpy", label="Upload ID Card"),
        gr.Image(type="numpy", label="Upload Selfie")
    ],
    outputs=[
        gr.Textbox(label="Verification Report", lines=20)
    ],
    title="AI-Powered KYC Verification Agent",
    description="Upload an ID card and a selfie. The system will extract data, verify the face, and generate a risk report."
)

iface.launch(debug=True)